# Check pnl across trades and positions

* Compare trade pnl between two subgraphs
* Compare position pnl between two subgraphs


In [45]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
import sqlite3
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px
from web3 import Web3
from web3.middleware import geth_poa_middleware

load_dotenv()

pio.renderers.default = 'notebook'


In [46]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# set up sqlite connection
con = sqlite3.connect("../../data/perps.db")

# get a web3 provider
w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))


In [47]:
# data contract
with open('../abi/PerpsV2Data.json', 'r') as file:
    PerpsV2DataAbi = json.dumps(json.load(file))

PerpsV2DataAddress = "0xF7D3D05cCeEEcC9d77864Da3DdE67Ce9a0215A9D"


## Get transfers and trades

In [48]:
df_trade = pd.read_sql_query('SELECT * FROM trades', con)
df_transfer = pd.read_sql_query('SELECT * FROM transfers', con)


In [49]:
## summarize transfers
df_transfer_summ = df_transfer.groupby('asset')['size'].sum().reset_index()
df_transfer_summ.columns = ['asset', 'netTransfers']

## summarize trades
df_trade_summ = df_trade.groupby('asset')['feesPaid'].sum().reset_index()
df_trade_summ.columns = ['asset', 'feesPaid']


In [50]:
df_trade_summ

,asset,feesPaid
0,AAVE,5088.440316
1,APE,30472.773200
2,ATOM,28051.559796
3,AUD,67.744650
4,AVAX,41719.046814
5,AXS,2643.551665
6,BNB,46505.526547
7,DOGE,10246.486298
8,DYDX,35475.158143
9,EUR,2024.181520


## Get the market data

In [51]:
perpsV2Data = w3.eth.contract(address=PerpsV2DataAddress, abi=PerpsV2DataAbi)
marketSummaries = perpsV2Data.functions.allProxiedMarketSummaries().call()



In [52]:
markets = [{
    'asset': market[1].decode().replace('\x00', ''),
    'marketDebt': w3.fromWei(market[7], unit='ether')
} for market in marketSummaries]

df_markets = pd.DataFrame(markets)
df_markets['marketDebt'] = df_markets['marketDebt'].astype(float)
df_markets

,asset,marketDebt
0,sETH,7.939714e+06
1,sBTC,9.209490e+06
2,LINK,1.786432e+05
3,SOL,1.929184e+05
4,AVAX,2.887931e+05
5,AAVE,1.752286e+03
6,UNI,8.376330e+04
7,MATIC,3.213194e+05
8,APE,5.965671e+04
9,DYDX,3.205192e+04


In [53]:
df_markets.dtypes

asset          object
marketDebt    float64
dtype: object

In [54]:
df_pnl = df_markets.merge(df_transfer_summ, on='asset').merge(df_trade_summ, on='asset')
df_pnl['netPnl'] = df_pnl['marketDebt'] - df_pnl['netTransfers'] - df_pnl['feesPaid']
df_pnl

,asset,marketDebt,netTransfers,feesPaid,netPnl
0,sETH,7.939714e+06,8.705478e+06,916987.829016,-1.682752e+06
1,sBTC,9.209490e+06,9.402499e+06,761420.491776,-9.544293e+05
2,LINK,1.786432e+05,2.621599e+05,90414.388551,-1.739311e+05
3,SOL,1.929184e+05,2.414476e+05,105936.201726,-1.544654e+05
4,AVAX,2.887931e+05,2.290636e+05,41719.046814,1.801042e+04
5,AAVE,1.752286e+03,-2.938926e+02,5088.440316,-3.042261e+03
6,UNI,8.376330e+04,9.547889e+04,10410.054421,-2.212564e+04
7,MATIC,3.213194e+05,4.018536e+05,84209.166874,-1.647433e+05
8,APE,5.965671e+04,3.086698e+04,30472.773200,-1.683038e+03
9,DYDX,3.205192e+04,5.040775e+04,35475.158143,-5.383099e+04


In [55]:
df_pnl.to_csv('output/market_upnl.csv', index=False)

In [56]:
df_pnl['netPnl'].sum()

-3730113.0490723196

In [57]:
df_pnl['netPnlClean'] = df_pnl['netPnl'].apply(lambda x: round(x, 2))
df_pnl[['asset', 'netPnlClean']].sort_values('netPnlClean', ascending=True)

,asset,netPnlClean
0,sETH,-1682752.35
1,sBTC,-954429.29
11,OP,-302047.43
2,LINK,-173931.06
7,MATIC,-164743.30
3,SOL,-154465.40
19,FTM,-115578.49
9,DYDX,-53830.99
16,ATOM,-45381.25
10,BNB,-35202.62
